---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

In [11]:
import re

# Function extract the date from a string
# Uses several regular expressions to look for dates in various formats
def find_date(datestring):
    months = {"Jan":"01","Feb":"02","Mar":"03","Apr":"04","May":"05","Jun":"06","Jul":"07","Aug":"08","Sep":"09",
              "Oct":"10","Nov":"11","Dec":"12"}
    m = re.search(r"(\d{1,2})[, \/-](\d{1,2})[, \/-]((\d{2}){1,3})",datestring)
    if m:
        return (('19' if len(m.group(3)) == 2 else '') + m.group(3) + ('0' if len(m.group(1)) == 1 else '') 
                + m.group(1) + ('0' if len(m.group(2)) == 1 else '') + m.group(2))
    
    m = re.search(r"(\d{1,2})[, \/-]+((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.]*)[, \/-]+((\d{2}){1,3})",datestring)
    if m:
        return (('19' if len(m.group(4)) == 2 else '') + m.group(4) + months[m.group(3)]
               + ('0' if len(m.group(1)) == 1 else '') + m.group(1))
    
    m = re.search(r"((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.]*)[, \/-]+(\d{1,2})(?:(?:th|st|nd)*)[, \/-]+((\d{2}){1,3})",datestring)
    #print(m.groups())
    if m:
        
        return (('19' if len(m.group(4)) == 2 else '') + m.group(4) + months[m.group(2)]
               + (('0' if len(m.group(3)) == 1 else '')  + m.group(3) if m.group(3) is not None else '01'))
    
    m = re.search(r"((Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z\.]*)[, \/-]+((\d{2}){1,3})",datestring)
    if m:
        
        return (('19' if len(m.group(3)) == 2 else '') + m.group(3) + months[m.group(2)]
               + '01')
    
    m = re.search(r"(\d{1,2})[, \/-]((\d{2}){1,3})",datestring)
    
    if m:
        
        return (('19' if len(m.group(2)) == 2 else '') + m.group(2) + ('0' if len(m.group(1)) == 1 else '') 
                + m.group(1)+'01')
    
    m = re.search(r"(\d{4})",datestring)
    if m:
        
        return (m.group(1)+'0101')
    
    else:
        return "0"

In [18]:
# We look a few sample sentences to extract the dates. 

for i in range(1,500,25):
    print("String: {0} --> Date: {1}".format(str.strip(df[i]),find_date(df[i])))

String: 6/18/85 Primary Care Doctor: --> Date: 19850618
String: 9/22/89 CPT Code: 90792: With medical services --> Date: 19890922
String: "In the context of the patient's work up, she had Neuropsychological  testing 5/24/91.  She was referred because of her cognitive decline and symptoms of motor dysfunction to help characterize her current functioning. : --> Date: 19910524
String: 12/5/87 Total time of visit (in minutes): --> Date: 19871205
String: 10/05/97 CPT Code: 90791: No medical services --> Date: 19971005
String: .10 Sep 2004 - Intake at EEC for IOP but did not follow up --> Date: 20040910
String: 22 June 1990 Medical History: --> Date: 19900622
String: see above and APS eval of 26 May 2010 Education Education Level: College Grad --> Date: 20100526
String: father was depressed inpatient at DFC December 23, 1999 Family History of Substance Abuse: Yes --> Date: 19991223
String: See previous - Parking garage incident January 07, 1991. --> Date: 19910107
String: December 1998 Prima

In [19]:
# Function to sort all the dates found in all the 500 lines of text and return a sorted order of indices
def date_sorter():
    datelist = pd.Series(pd.Series(data=list(map(find_date,df))).sort_values().index)
    return datelist           
date_sorter().head(20)

0       9
1      84
2       2
3      53
4      28
5     474
6     153
7      13
8     129
9      98
10    111
11    225
12     31
13    171
14    191
15    486
16    335
17    415
18     36
19    405
dtype: int64